In [3]:
import numpy as np
import pandas as pd

In [4]:
id_ls=[i for i in range(1,16)]
age_ls=[1,1,1,1,1 ,2,2,2,2,2, 3,3,3,3,3] #1青年2中年3老年
work_ls=[0,0,1,1,0, 0,0,1,0,0, 0,0,1,1,0] #1是0否
house_ls=[0,0,0,1,0, 0,0,1,1,1, 1,1,0,0,0] #1是0否
loan_ls=[1,2,2,1,1, 1,2,2,3,3, 3,2,2,3,1] #1一般，2好，3非常好
kind_ls=[0,0,1,1,0, 0,0,1,1,1, 1,1,1,1,0] #1是0否
data_set=pd.DataFrame({"id":id_ls,"age":age_ls,"work":work_ls,"house":house_ls,"loan":loan_ls,"kind":kind_ls})

In [5]:
def h(k_ls): #计算信息熵
    res=0
    for i in k_ls:
        if i==0 or i==1:
            pass
        else:
            res+=i*np.log2(i)
    return -res
def h_odds(k_ls,k_odds): #计算条件熵
    res=0
    for p,p_odd in zip(k_ls,k_odds):
        res+=p*h([p_odd,1-p_odd])
    return res
def count_A(data):
    res=[]
    sum=data.__len__()
    p_sum=(data["kind"]==1).sum()
    h_d=h([p_sum/sum,1-p_sum/sum])
    for i in data.keys():
        p_odd=[]
        p_odd_p=[]
        if i=="id" or i=="kind":
            pass
        else:
            A_k=data[i].unique()
            for j in A_k:
                p_odd.append((data[i]==j).sum()/sum)
                _p=(data[data[i]==j]["kind"]==1).sum() / (data[i]==j).sum()
                p_odd_p.append(_p)
            gain_ = h_d - h_odds(p_odd,p_odd_p)
            iv_ = h(p_odd)
            #print(f"属性{i}的为{iv_}")
            if(iv_==0): #势必为上个节点
                gain_ratio_=0
            else:
                gain_ratio_ = gain_ / iv_
            res.append(gain_ratio_)
            #print("@")
            print(f"属性{i}的增益率为{gain_ratio_}")
    return res
def max_A(data):
    max_a=np.argmax(data)#最大增益率对应的属性
    res_ls=["age","work","house","loan"]
    return res_ls[max_a]
res=count_A(data_set)
max_a=max_A(res)#最大增益率对应的属性
def split_data(data,max_a):
    res_ls=[]
    A_k=data[max_a].unique()
    for i in A_k:
        res=data[data[max_a]==i]
        res_ls.append(res)
        res_id=res["id"].tolist()
        print(f"@@@划分：{res_id}")
        #测试
    return res_ls


属性age的增益率为0.05237190142858302
属性work的增益率为0.3524465495205019
属性house的增益率为0.4325380677663126
属性loan的增益率为0.23185388128724224


In [7]:
#第一次划分
s1=split_data(data_set,max_a)
print("===========")
#第二次划分
s2_1_A=max_A(count_A(s1[0])) #左
#s2_2_A=max_A(count_A(s1[1])) #右--为一类
s2_1=split_data(s1[0],s2_1_A)  #分支2岔
print("===========")
#s2_2=split_data(s1[1],s2_2_A) #分支3岔
print("===========")
#先处理分支2
s2_1[0]["kind"].unique().size

@@@划分：[1, 2, 3, 5, 6, 7, 13, 14, 15]
@@@划分：[4, 8, 9, 10, 11, 12]
属性age的增益率为0.1644105252727686
属性work的增益率为1.0
属性house的增益率为0
属性loan的增益率为0.34037448163185724
@@@划分：[1, 2, 5, 6, 7, 15]
@@@划分：[3, 13, 14]


1

In [49]:

age=[2/5,3/5,4/5]
age_p=[1/3,1/3,1/3]

work=[1,4/10]
work_p=[1/3,2/3]

house=[1,3/9]
house_p=[2/5,3/5]

loan=[1/5,4/6,1]
loan_p=[1/3,3/5,4/15]
#print(h(age))
k=[9/15,6/15]
h_D=h(k)
def h_odds(k_ls,k_odds): #计算条件熵
    res=0
    for p,p_odd in zip(k_ls,k_odds):
        res+=p*h([p_odd,1-p_odd])
    return res
h_D_age=h_odds(age_p,age) #计算年龄的条件熵
gain_age=h_D-h_D_age #计算信息增益
IV_age=h(age_p)
print(f"年龄的信息增益:{gain_age} | 年龄的信息增益率:{gain_age/IV_age}")

h_D_work=h_odds(work_p,work)
gain_work=h_D-h_D_work
IV_work=h(work_p)
print(f"有工作的信息增益:{gain_work} | 有工作的信息增益率:{gain_work/IV_work}")

h_D_house=h_odds(house_p,house)
gain_house=h_D-h_D_house
IV_house=h(house_p)
print(f"有房子的信息增益:{gain_house} | 有房子的信息增益率:{gain_house/IV_house}")

h_D_loan=h_odds(loan_p,loan)
gain_loan=h_D-h_D_loan
IV_loan=h(loan_p)
print(f"有贷款的信息增益:{gain_loan} | 有贷款的信息增益率:{gain_loan/IV_loan}")

年龄的信息增益:0.08300749985576883 | 年龄的信息增益率:0.05237190142858302
有工作的信息增益:0.32365019815155627 | 有工作的信息增益率:0.3524465495205019
有房子的信息增益:0.4199730940219749 | 有房子的信息增益率:0.4325380677663126
有贷款的信息增益:0.17933039572618747 | 有贷款的信息增益率:0.121250756199111


In [9]:
x=[i for i in range(1,11)]
y=[4.50, 4.75, 4.91, 5.34, 5.80, 7.05 ,7.90, 8.23, 8.70, 9.00]
data_set=pd.DataFrame({"x":x,"y":y})
data_set

,x,y
0,1,4.50
1,2,4.75
2,3,4.91
3,4,5.34
4,5,5.80
5,6,7.05
6,7,7.90
7,8,8.23
8,9,8.70
9,10,9.00


In [20]:
def mse(data,s):
    data1=data[data["x"]<=s]
    data2=data[data["x"]>s]
    res=0
    c1= data1["y"].sum()/data1["y"].size #均值
    if data2["y"].size==0:
        c2=0
    else:
        c2= data2["y"].sum()/data2["y"].size
    for i in data1["y"]:
        res+=(i-c1)**2
    for i in data2["y"]:
        res+=(i-c2)**2
    #print(pd.DataFrame([c1,c2]))
    print(f"{s}:mse：{res}")
    return res
res_=[]
for i in data_set["x"]:
    mse_res=mse(data_set,i)
    #print(mse_res,end=" | ")
    res_.append(mse_res)

def mse_clip(data):
    res_=[]
    for i in data["x"]:
        mse_res=mse(data,i)
        res_.append(mse_res)
    ans=np.argmin(res_)
    ans_= data["x"].tolist()[ans] 
    print("最佳划分点为",ans_)
    d1=data[data["x"]<=ans_]
    d2=data[data["x"]>ans_]
    return d1,d2
print("===========")
d1,d2=mse_clip(data_set)
#显然，取5的时候，mse最小，为最佳划分点

1:mse：22.647999999999996
2:mse：17.7022375
3:mse：12.193485714285714
4:mse：7.3787
5:mse：3.358719999999999
6:mse：5.073958333333332
7:mse：10.052466666666668
8:mse：15.177800000000005
9:mse：21.328
10:mse：27.63236
1:mse：22.647999999999996
2:mse：17.7022375
3:mse：12.193485714285714
4:mse：7.3787
5:mse：3.358719999999999
6:mse：5.073958333333332
7:mse：10.052466666666668
8:mse：15.177800000000005
9:mse：21.328
10:mse：27.63236
最佳划分点为 5


In [23]:
print("第一次划分结果",d1["x"].to_list(),"和",d2["x"].to_list())
print("对d1进行划分")
d1_1,d1_2=mse_clip(d1)
print("d1划分结果:",d1_1["x"].to_list(),"和",d1_2["x"].to_list())
print("===========")
print("对d1_1进行划分")
d1_1_1,d1_1_2=mse_clip(d1_1)
print("d1_1划分结果:",d1_1_1["x"].to_list(),"和",d1_1_2["x"].to_list())
print("@@@d1_1_1划分结束")


第一次划分结果 [1, 2, 3, 4, 5] 和 [6, 7, 8, 9, 10]
对d1进行划分
1:mse：0.6661999999999997
2:mse：0.4274499999999997
3:mse：0.19120000000000004
4:mse：0.37369999999999987
5:mse：1.0581999999999996
最佳划分点为 3
d1划分结果: [1, 2, 3] 和 [4, 5]
对d1_1进行划分
1:mse：0.012800000000000023
2:mse：0.03125
3:mse：0.08540000000000005
最佳划分点为 1
d1_1划分结果: [1] 和 [2, 3]
@@@d1_1_1划分结束
对d1_1_2进行划分
2:mse：0.0
3:mse：0.012800000000000023
最佳划分点为 2
d1_1_2划分结果: [2] 和 [3]
@@@@d1_1_2划分结束
对d2进行划分
6:mse：0.7156749999999991
7:mse：0.6625166666666666
8:mse：0.7862666666666676
9:mse：1.4517999999999995
10:mse：2.3005199999999992
最佳划分点为 7
d2划分结果: [6, 7] 和 [8, 9, 10]
对d1_2进行划分
4:mse：0.0
5:mse：0.10579999999999998
最佳划分点为 4
d1_2划分结果: [4] 和 [5]
@@@@d1_2划分结束


In [24]:
print("===========")
print("对d1_1_2进行划分")
d1_1_2_1,d1_1_2_2=mse_clip(d1_1_2)
print("d1_1_2划分结果:",d1_1_2_1["x"].to_list(),"和",d1_1_2_2["x"].to_list())
print("@@@@d1_1_2划分结束")
print("===========")
print("对d2进行划分")
d2_1,d2_2=mse_clip(d2)
print("d2划分结果:",d2_1["x"].to_list(),"和",d2_2["x"].to_list())
print("===========")
print("对d1_2进行划分")
d1_2_1,d1_2_2=mse_clip(d1_2)
print("d1_2划分结果:",d1_2_1["x"].to_list(),"和",d1_2_2["x"].to_list())
print("@@@@d1_2划分结束")
print("===========")

对d1_1_2进行划分
2:mse：0.0
3:mse：0.012800000000000023
最佳划分点为 2
d1_1_2划分结果: [2] 和 [3]
@@@@d1_1_2划分结束
对d2进行划分
6:mse：0.7156749999999991
7:mse：0.6625166666666666
8:mse：0.7862666666666676
9:mse：1.4517999999999995
10:mse：2.3005199999999992
最佳划分点为 7
d2划分结果: [6, 7] 和 [8, 9, 10]
对d1_2进行划分
4:mse：0.0
5:mse：0.10579999999999998
最佳划分点为 4
d1_2划分结果: [4] 和 [5]
@@@@d1_2划分结束


In [22]:

print("对d2_1进行划分")
d2_1_1,d2_1_2=mse_clip(d2_1)
print("d2_1划分结果:",d2_1_1["x"].to_list(),"和",d2_1_2["x"].to_list())
print("@@@@d2_1划分结束")
print("===========")
print("对d2_2进行划分")
d2_2_1,d2_2_2=mse_clip(d2_2)
print("d2_2划分结果:",d2_2_1["x"].to_list(),"和",d2_2_2["x"].to_list())
print("!!!到这里其实就很显然了")

对d2_1进行划分
6:mse：0.0
7:mse：0.3612500000000004
最佳划分点为 6
d2_1划分结果: [6] 和 [7]
@@@@d2_1划分结束
对d2_2进行划分
8:mse：0.04500000000000021
9:mse：0.11044999999999947
10:mse：0.30126666666666624
最佳划分点为 8
d2_2划分结果: [8] 和 [9, 10]
!!!到这里其实就很显然了
